In [1]:
import spacy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
import os
import glob
import io
import random
SEED = 2206
torch.manual_seed(SEED)

In [3]:
nlp = spacy.load('en')
'''

# Clean text before feeding it to spaCy
punctuations = string.punctuation
stopwords = stopwords.words('english')

# Define function to cleanup text by removing personal pronouns, stopwords, and punctuation
def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser','ner'])
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)

def remove_s(text):
    word_clean=List[Any]=[word for word in text if word != '\s']
    return word_clean
# Parse documents during words into vectors
def parse_documents(cleaned_text):
    print('Parsing documents...')
    train_vec = []
    for doc in nlp.pipe(cleaned_text, batch_size=500):
        if doc.has_vector:
            train_vec.append(doc.vector)
            #print(len(doc.vector))
        # If doc doesn't have a vector, then fill it with zeros.
        else:
            train_vec.append(np.zeros((300,), dtype="float32"))
    train_vec = np.array(train_vec)
    #print(len(train_vec))
    #sys.exit(1)
    return train_vec
'''

'\n\n# Clean text before feeding it to spaCy\npunctuations = string.punctuation\nstopwords = stopwords.words(\'english\')\n\n# Define function to cleanup text by removing personal pronouns, stopwords, and punctuation\ndef cleanup_text(docs, logging=False):\n    texts = []\n    counter = 1\n    for doc in docs:\n        if counter % 1000 == 0 and logging:\n            print("Processed %d out of %d documents." % (counter, len(docs)))\n        counter += 1\n        doc = nlp(doc, disable=[\'parser\',\'ner\'])\n        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != \'-PRON-\']\n        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]\n        tokens = \' \'.join(tokens)\n        texts.append(tokens)\n    return pd.Series(texts)\n\ndef remove_s(text):\n    word_clean=List[Any]=[word for word in text if word != \'\\s\']\n    return word_clean\n# Parse documents during words into vectors\ndef parse_documents(cleaned_text):\n    print(\

In [4]:
TEXT = data.Field(lower = True, include_lengths = True, batch_first = True)
LABEL = data.LabelField(sequential = False)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)


aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:04<00:00, 20.9MB/s]


In [5]:
print('train_data.fields', train_data.fields)
print('len(train_data)', len(train_data))
print('vars(train_data)', vars(train_data[0]))


train_data.fields {'text': <torchtext.data.field.Field object at 0x7f9991c93fd0>, 'label': <torchtext.data.field.LabelField object at 0x7f9991c93588>}
len(train_data) 25000
vars(train_data) {'text': ['an', "insomniac's", 'nightmare', 'is', 'the', 'story', 'of', 'a', "man's", 'plunge', 'into', 'insanity.', 'having', 'chronic', 'insomnia,', 'jack', 'is', 'plagued', 'by', 'hallucinations;', 'causing', 'him', 'to', 'try', 'and', 'determine', 'what', 'is', 'real', 'and', 'what', "isn't.<br", '/><br', '/>we', 'find', 'out', 'interesting', 'things', 'about', 'jack', 'near', 'the', 'end,', 'and', 'think', 'that', 'by', 'the', 'time', 'the', 'movie', 'is', 'over', 'we', 'will', 'have', 'a', '"happily', 'ever', 'after"', 'hollywood', 'ending.', 'wrong.', 'this', 'is', 'new', 'york', 'city,', 'the', 'place', 'where', 'nobody', 'sleeps.<br', '/><br', '/>tess', 'nanavati', '(writer', 'and', 'director)', 'has', 'herself', 'a', 'good', 'film', 'in', "'an", "insomniac's", "nightmare'.", 'a', 'talented

In [0]:
train_data, validation_data = train_data.split(random_state=random.seed(SEED), split_ratio=.8)

In [9]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(f'Number of validation examples: {len(validation_data)}')

Number of training examples: 16000
Number of testing examples: 25000
Number of validation examples: 4000


In [10]:
# Building Vocab
TEXT.build_vocab(train_data, vectors =  GloVe(name = '6B', dim =  300), max_size=25000)
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [01:55, 7.46MB/s]                           
100%|█████████▉| 399316/400000 [00:48<00:00, 8382.73it/s]

In [20]:
BATCH_SIZE = 3

device = 'cuda:0'

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, validation_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

batch = next(iter(train_iterator))
print(batch.text)
print(batch.label)

(tensor([[ 4439,     2,   117,  ...,    11,   313, 15926],
        [  102,    66,    10,  ...,     1,     1,     1],
        [   10,    20,    14,  ...,     1,     1,     1]], device='cuda:0'), tensor([473,  81,  37], device='cuda:0'))
tensor([0, 1, 1], device='cuda:0')


In [0]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        #self.relu1 = nn.ReLU()
        
        
    def forward(self, text):

        # text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        # embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        # output = [sent len, batch size, hid dim]
        # hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        hidden2 = self.fc1(hidden.squeeze(0))
        
        return self.fc2(hidden2)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)
# accuracy function
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, 
    this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

# train function
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# evaluate accuracy
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [31]:
# train model
N_EPOCHS = 1

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | \
          Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | \
          Val. Acc: {valid_acc*100:.2f}% |')


AttributeError: ignored